In [1]:
import pandas as pd
import tensorflow as tf
import numpy
target = pd.read_csv('train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id').apply(lambda x: x-1)
df_train_values_original = pd.read_csv('train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')


pd.options.display.float_format = '{:20,.2f}'.format

In [2]:
df = df_train_values_original.drop(columns=["geo_level_1_id", "geo_level_2_id", "geo_level_3_id"])

In [3]:
def one_hot_encode_data(dataframe, column_name):
    dummies = pd.get_dummies(dataframe[column_name])
    rename_columns = {}
    for column in dummies.columns.values:
        rename_columns[column] = column_name + '_' + column
    return dataframe.drop(columns=[column_name]).join(dummies.rename(columns=rename_columns))

In [4]:
df = one_hot_encode_data(df,'land_surface_condition')
df = one_hot_encode_data(df,'foundation_type')
df = one_hot_encode_data(df,'roof_type')
df = one_hot_encode_data(df,'ground_floor_type')
df = one_hot_encode_data(df,'other_floor_type')
df = one_hot_encode_data(df,'position')
df = one_hot_encode_data(df,'plan_configuration')
df = one_hot_encode_data(df,'legal_ownership_status')
df

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
28830,2,10,8,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
94947,2,10,5,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
590882,2,10,6,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
201944,3,30,8,9,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,1,55,6,3,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
669485,2,0,6,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
602512,3,55,6,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [5]:
dataset = tf.data.Dataset.from_tensor_slices((df[:int(len(df)/1)].values, target[:int(len(df)/1)].values))
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))


Features: [ 2 30  6  5  1  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
  0  0  0  0  0  1  0  0  1  0  0  1  0  0  1  0  0  0  0  0  1  0  0  0
  0  0  1  0  0  1  0  0  0  0  0  0  0  0  0  1  0], Target: [2]
Features: [ 2 10  8  7  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
  0  0  0  0  1  0  0  0  1  0  0  1  0  0  0  0  0  1  0  0  1  0  0  0
  0  1  0  0  0  1  0  0  0  0  0  0  0  0  0  1  0], Target: [1]
Features: [ 2 10  5  5  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
  0  0  0  0  0  1  0  0  1  0  0  1  0  0  1  0  0  0  0  0  0  0  1  0
  0  0  1  0  0  1  0  0  0  0  0  0  0  0  0  1  0], Target: [2]
Features: [ 2 10  6  5  0  1  0  0  0  0  1  1  0  0  0  1  0  0  0  0  0  0  0  0
  0  0  0  0  0  1  0  0  1  0  0  1  0  0  1  0  0  0  0  0  0  0  1  0
  0  1  0  0  0  1  0  0  0  0  0  0  0  0  0  1  0], Target: [1]
Features: [ 3 30  8  9  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
  0  0  0  0  0  1  0  0  1  

In [6]:
def compile_model(l_size, l_amount, activation):
    layers = []
    for x in range(l_amount):
        layers.append(tf.keras.layers.Dense(l_size, activation=activation))
    layers.append(tf.keras.layers.Dense(units=3, activation='softmax'))
    model = tf.keras.Sequential(layers)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    model.compile(optimizer='Adagrad',
                  loss=loss_fn,
                  metrics=['accuracy'])
    return model

In [7]:
grid_search = [
    [130, 195],
    [3, 4],
    ['relu', 'sigmoid']
]
results = []
with tf.device('/CPU:0'):
    for n in grid_search[1]:
        for i in grid_search[0]:
            for a in grid_search[2]:
                m = compile_model(i, n, a)
                m.fit(dataset.batch(32), epochs=50)
                results.append(m.evaluate(df.to_numpy(), target.to_numpy()))

Epoch 1/50
8144/8144 [==============================] - 9s 1ms/step - loss: 0.8501 - accuracy: 0.5714
Epoch 2/50
8144/8144 [==============================] - 8s 1ms/step - loss: 0.8138 - accuracy: 0.5781
Epoch 3/50
8144/8144 [==============================] - 8s 1ms/step - loss: 0.8058 - accuracy: 0.5806
Epoch 4/50
8144/8144 [==============================] - 8s 1ms/step - loss: 0.8022 - accuracy: 0.5823
Epoch 5/50
8144/8144 [==============================] - 8s 955us/step - loss: 0.8000 - accuracy: 0.5838
Epoch 6/50
8144/8144 [==============================] - 8s 964us/step - loss: 0.7983 - accuracy: 0.5848
Epoch 7/50
8144/8144 [==============================] - 8s 967us/step - loss: 0.7970 - accuracy: 0.5861
Epoch 8/50
8144/8144 [==============================] - 8s 949us/step - loss: 0.7959 - accuracy: 0.5872
Epoch 9/50
8144/8144 [==============================] - 8s 934us/step - loss: 0.7950 - accuracy: 0.5883
Epoch 10/50
8144/8144 [==============================] - 8s 923us/step -

8144/8144 [==============================] - 8s 956us/step - loss: 0.8165 - accuracy: 0.5755
Epoch 29/50
8144/8144 [==============================] - 8s 942us/step - loss: 0.8154 - accuracy: 0.5756
Epoch 30/50
8144/8144 [==============================] - 8s 948us/step - loss: 0.8144 - accuracy: 0.5758
Epoch 31/50
8144/8144 [==============================] - 8s 1ms/step - loss: 0.8135 - accuracy: 0.5760
Epoch 32/50
8144/8144 [==============================] - 9s 1ms/step - loss: 0.8128 - accuracy: 0.5760
Epoch 33/50
8144/8144 [==============================] - 8s 1ms/step - loss: 0.8121 - accuracy: 0.5761
Epoch 34/50
8144/8144 [==============================] - 9s 1ms/step - loss: 0.8115 - accuracy: 0.5761
Epoch 35/50
8144/8144 [==============================] - 9s 1ms/step - loss: 0.8109 - accuracy: 0.5763
Epoch 36/50
8144/8144 [==============================] - 8s 1ms/step - loss: 0.8104 - accuracy: 0.5763
Epoch 37/50
8144/8144 [==============================] - 8s 1ms/step - loss: 0.

8144/8144 [==============================] - 10s 1ms/step - loss: 0.8722 - accuracy: 0.5689
Epoch 6/50
8144/8144 [==============================] - 11s 1ms/step - loss: 0.8701 - accuracy: 0.5694
Epoch 7/50
8144/8144 [==============================] - 11s 1ms/step - loss: 0.8681 - accuracy: 0.5698
Epoch 8/50
8144/8144 [==============================] - 11s 1ms/step - loss: 0.8660 - accuracy: 0.5702
Epoch 9/50
8144/8144 [==============================] - 12s 1ms/step - loss: 0.8638 - accuracy: 0.5709
Epoch 10/50
8144/8144 [==============================] - 12s 1ms/step - loss: 0.8614 - accuracy: 0.5713
Epoch 11/50
8144/8144 [==============================] - 10s 1ms/step - loss: 0.8589 - accuracy: 0.5716
Epoch 12/50
8144/8144 [==============================] - 11s 1ms/step - loss: 0.8561 - accuracy: 0.5721
Epoch 13/50
8144/8144 [==============================] - 11s 1ms/step - loss: 0.8532 - accuracy: 0.5724
Epoch 14/50
8144/8144 [==============================] - 10s 1ms/step - loss: 0.

8144/8144 [==============================] - 10s 1ms/step - loss: 0.7869 - accuracy: 0.5949
Epoch 34/50
8144/8144 [==============================] - 10s 1ms/step - loss: 0.7867 - accuracy: 0.5950
Epoch 35/50
8144/8144 [==============================] - 10s 1ms/step - loss: 0.7866 - accuracy: 0.5952
Epoch 36/50
8144/8144 [==============================] - 9s 1ms/step - loss: 0.7865 - accuracy: 0.5954
Epoch 37/50
8144/8144 [==============================] - 10s 1ms/step - loss: 0.7863 - accuracy: 0.5957
Epoch 38/50
8144/8144 [==============================] - 9s 1ms/step - loss: 0.7862 - accuracy: 0.5958
Epoch 39/50
8144/8144 [==============================] - 9s 1ms/step - loss: 0.7861 - accuracy: 0.5959
Epoch 40/50
8144/8144 [==============================] - 10s 1ms/step - loss: 0.7860 - accuracy: 0.5959
Epoch 41/50
8144/8144 [==============================] - 10s 1ms/step - loss: 0.7859 - accuracy: 0.5960
Epoch 42/50
8144/8144 [==============================] - 10s 1ms/step - loss: 0

8144/8144 [==============================] - 13s 2ms/step - loss: 0.7911 - accuracy: 0.5912
Epoch 11/50
8144/8144 [==============================] - 13s 2ms/step - loss: 0.7905 - accuracy: 0.5916
Epoch 12/50
8144/8144 [==============================] - 14s 2ms/step - loss: 0.7900 - accuracy: 0.5921
Epoch 13/50
8144/8144 [==============================] - 15s 2ms/step - loss: 0.7895 - accuracy: 0.5926
Epoch 14/50
8144/8144 [==============================] - 14s 2ms/step - loss: 0.7891 - accuracy: 0.5929
Epoch 15/50
8144/8144 [==============================] - 16s 2ms/step - loss: 0.7887 - accuracy: 0.5932
Epoch 16/50
8144/8144 [==============================] - 15s 2ms/step - loss: 0.7884 - accuracy: 0.5934
Epoch 17/50
8144/8144 [==============================] - 16s 2ms/step - loss: 0.7881 - accuracy: 0.5936
Epoch 18/50
8144/8144 [==============================] - 14s 2ms/step - loss: 0.7878 - accuracy: 0.5938
Epoch 19/50
8144/8144 [==============================] - 14s 2ms/step - loss

8144/8144 [==============================] - 13s 2ms/step - loss: 0.8167 - accuracy: 0.5749
Epoch 39/50
8144/8144 [==============================] - 12s 1ms/step - loss: 0.8157 - accuracy: 0.5749
Epoch 40/50
8144/8144 [==============================] - 12s 1ms/step - loss: 0.8149 - accuracy: 0.5750
Epoch 41/50
8144/8144 [==============================] - 12s 1ms/step - loss: 0.8141 - accuracy: 0.5751
Epoch 42/50
8144/8144 [==============================] - 12s 1ms/step - loss: 0.8134 - accuracy: 0.5752
Epoch 43/50
8144/8144 [==============================] - 12s 1ms/step - loss: 0.8127 - accuracy: 0.5753
Epoch 44/50
8144/8144 [==============================] - 12s 1ms/step - loss: 0.8121 - accuracy: 0.5754
Epoch 45/50
8144/8144 [==============================] - 12s 1ms/step - loss: 0.8116 - accuracy: 0.5754
Epoch 46/50
8144/8144 [==============================] - 12s 2ms/step - loss: 0.8110 - accuracy: 0.5756
Epoch 47/50
8144/8144 [==============================] - 13s 2ms/step - loss

In [8]:
print(results)

[[0.7856335639953613, 0.5955042243003845], [0.8056017756462097, 0.5788772702217102], [0.7835327982902527, 0.5982900857925415], [0.8047553896903992, 0.5798404216766357], [0.7845270037651062, 0.5968703031539917], [0.8122333288192749, 0.5754237174987793], [0.7827788591384888, 0.5981979966163635], [0.808894157409668, 0.5760338306427002]]


In [12]:
j = 0
for n in grid_search[1]:
    for i in grid_search[0]:
        for a in grid_search[2]:
            print(results[j][1])
            j+=1
for n in grid_search[1]:
    for i in grid_search[0]:
        for a in grid_search[2]:
            print(str(n)+','+str(i)+','+str(a))

0.5955042243003845
0.5788772702217102
0.5982900857925415
0.5798404216766357
0.5968703031539917
0.5754237174987793
0.5981979966163635
0.5760338306427002
3,130,relu
3,130,sigmoid
3,195,relu
3,195,sigmoid
4,130,relu
4,130,sigmoid
4,195,relu
4,195,sigmoid


In [10]:
results[0][1]

0.5955042243003845

In [14]:
best_model = compile_model(195, 4, 'relu')
best_model.fit(dataset.batch(32), epochs=1)

8144/8144 [==============================] - 27s 3ms/step - loss: 0.8405 - accuracy: 0.5745


In [15]:
best_model.fit(dataset.batch(32), epochs=1000)

Epoch 1/1000
8144/8144 [==============================] - 28s 3ms/step - loss: 0.8086 - accuracy: 0.5807
Epoch 2/1000
8144/8144 [==============================] - 26s 3ms/step - loss: 0.8019 - accuracy: 0.5839
Epoch 3/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7987 - accuracy: 0.5859
Epoch 4/1000
8144/8144 [==============================] - 27s 3ms/step - loss: 0.7965 - accuracy: 0.5871
Epoch 5/1000
8144/8144 [==============================] - 27s 3ms/step - loss: 0.7950 - accuracy: 0.5884
Epoch 6/1000
8144/8144 [==============================] - 27s 3ms/step - loss: 0.7938 - accuracy: 0.5898
Epoch 7/1000
8144/8144 [==============================] - 28s 3ms/step - loss: 0.7928 - accuracy: 0.5905
Epoch 8/1000
8144/8144 [==============================] - 28s 3ms/step - loss: 0.7920 - accuracy: 0.5913
Epoch 9/1000
8144/8144 [==============================] - 29s 4ms/step - loss: 0.7913 - accuracy: 0.5920
Epoch 10/1000
8144/8144 [==============================

8144/8144 [==============================] - 28s 3ms/step - loss: 0.7811 - accuracy: 0.6005
Epoch 79/1000
8144/8144 [==============================] - 28s 3ms/step - loss: 0.7811 - accuracy: 0.6005
Epoch 80/1000
8144/8144 [==============================] - 28s 3ms/step - loss: 0.7810 - accuracy: 0.6005
Epoch 81/1000
8144/8144 [==============================] - 27s 3ms/step - loss: 0.7809 - accuracy: 0.6005
Epoch 82/1000
8144/8144 [==============================] - 28s 3ms/step - loss: 0.7809 - accuracy: 0.6005
Epoch 83/1000
8144/8144 [==============================] - 26s 3ms/step - loss: 0.7808 - accuracy: 0.6006
Epoch 84/1000
8144/8144 [==============================] - 26s 3ms/step - loss: 0.7807 - accuracy: 0.6007
Epoch 85/1000
8144/8144 [==============================] - 27s 3ms/step - loss: 0.7807 - accuracy: 0.6008
Epoch 86/1000
8144/8144 [==============================] - 29s 3ms/step - loss: 0.7806 - accuracy: 0.6009
Epoch 87/1000
8144/8144 [==============================] - 2

8144/8144 [==============================] - 27s 3ms/step - loss: 0.7772 - accuracy: 0.6034
Epoch 155/1000
8144/8144 [==============================] - 27s 3ms/step - loss: 0.7771 - accuracy: 0.6035
Epoch 156/1000
8144/8144 [==============================] - 27s 3ms/step - loss: 0.7771 - accuracy: 0.6036
Epoch 157/1000
8144/8144 [==============================] - 27s 3ms/step - loss: 0.7771 - accuracy: 0.6037
Epoch 158/1000
8144/8144 [==============================] - 27s 3ms/step - loss: 0.7770 - accuracy: 0.6037
Epoch 159/1000
8144/8144 [==============================] - 27s 3ms/step - loss: 0.7770 - accuracy: 0.6038
Epoch 160/1000
8144/8144 [==============================] - 27s 3ms/step - loss: 0.7769 - accuracy: 0.6038
Epoch 161/1000
8144/8144 [==============================] - 27s 3ms/step - loss: 0.7769 - accuracy: 0.6039
Epoch 162/1000
8144/8144 [==============================] - 27s 3ms/step - loss: 0.7769 - accuracy: 0.6038
Epoch 163/1000
8144/8144 [==========================

8144/8144 [==============================] - 25s 3ms/step - loss: 0.7745 - accuracy: 0.6052
Epoch 231/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7744 - accuracy: 0.6052
Epoch 232/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7744 - accuracy: 0.6053
Epoch 233/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7744 - accuracy: 0.6053
Epoch 234/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7743 - accuracy: 0.6053
Epoch 235/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7743 - accuracy: 0.6053
Epoch 236/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7743 - accuracy: 0.6053
Epoch 237/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7743 - accuracy: 0.6053
Epoch 238/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7742 - accuracy: 0.6054
Epoch 239/1000
8144/8144 [==========================

8144/8144 [==============================] - 25s 3ms/step - loss: 0.7724 - accuracy: 0.6068
Epoch 307/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7724 - accuracy: 0.6068
Epoch 308/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7723 - accuracy: 0.6068
Epoch 309/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7723 - accuracy: 0.6069
Epoch 310/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7723 - accuracy: 0.6069
Epoch 311/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7723 - accuracy: 0.6069
Epoch 312/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7722 - accuracy: 0.6069
Epoch 313/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7722 - accuracy: 0.6070
Epoch 314/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7722 - accuracy: 0.6070
Epoch 315/1000
8144/8144 [==========================

...

8144/8144 [==============================] - 25s 3ms/step - loss: 0.7659 - accuracy: 0.6119
Epoch 687/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7658 - accuracy: 0.6118
Epoch 688/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7658 - accuracy: 0.6119
Epoch 689/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7658 - accuracy: 0.6119
Epoch 690/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7658 - accuracy: 0.6119
Epoch 691/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7658 - accuracy: 0.6119
Epoch 692/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7658 - accuracy: 0.6119
Epoch 693/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7658 - accuracy: 0.6119
Epoch 694/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7658 - accuracy: 0.6119
Epoch 695/1000
8144/8144 [==========================

8144/8144 [==============================] - 25s 3ms/step - loss: 0.7649 - accuracy: 0.6124
Epoch 763/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7649 - accuracy: 0.6125
Epoch 764/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7649 - accuracy: 0.6125
Epoch 765/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7649 - accuracy: 0.6125
Epoch 766/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7648 - accuracy: 0.6125
Epoch 767/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7648 - accuracy: 0.6126
Epoch 768/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7648 - accuracy: 0.6126
Epoch 769/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7648 - accuracy: 0.6126
Epoch 770/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7648 - accuracy: 0.6126
Epoch 771/1000
8144/8144 [==========================

8144/8144 [==============================] - 25s 3ms/step - loss: 0.7640 - accuracy: 0.6131
Epoch 839/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7640 - accuracy: 0.6130
Epoch 840/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7640 - accuracy: 0.6131
Epoch 841/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7640 - accuracy: 0.6131
Epoch 842/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7640 - accuracy: 0.6131
Epoch 843/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7640 - accuracy: 0.6131
Epoch 844/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7639 - accuracy: 0.6131
Epoch 845/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7639 - accuracy: 0.6132
Epoch 846/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7639 - accuracy: 0.6132
Epoch 847/1000
8144/8144 [==========================

8144/8144 [==============================] - 25s 3ms/step - loss: 0.7632 - accuracy: 0.6138
Epoch 915/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7632 - accuracy: 0.6138
Epoch 916/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7632 - accuracy: 0.6138
Epoch 917/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7631 - accuracy: 0.6139
Epoch 918/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7631 - accuracy: 0.6139
Epoch 919/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7631 - accuracy: 0.6139
Epoch 920/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7631 - accuracy: 0.6139
Epoch 921/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7631 - accuracy: 0.6139
Epoch 922/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7631 - accuracy: 0.6139
Epoch 923/1000
8144/8144 [==========================

8144/8144 [==============================] - 25s 3ms/step - loss: 0.7624 - accuracy: 0.6146
Epoch 991/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7624 - accuracy: 0.6145
Epoch 992/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7624 - accuracy: 0.6146
Epoch 993/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7624 - accuracy: 0.6146
Epoch 994/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7623 - accuracy: 0.6146
Epoch 995/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7623 - accuracy: 0.6146
Epoch 996/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7623 - accuracy: 0.6146
Epoch 997/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7623 - accuracy: 0.6146
Epoch 998/1000
8144/8144 [==============================] - 25s 3ms/step - loss: 0.7623 - accuracy: 0.6146
Epoch 999/1000
8144/8144 [==========================

In [16]:
best_model.save('modelos/NNModelGS')

INFO:tensorflow:Assets written to: modelos/NNModelGS\assets
